In [1]:
!pip install transformers datasets torch


In [2]:
!pip install transformers
!pip install torch

from transformers import BartForConditionalGeneration, BartTokenizer
import torch


c:\Users\Prathiksha\AppData\Local\Programs\Python\Python310\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
import json
import pandas as pd
from datasets import Dataset

# Load the JSON data
with open('intent 400.json') as f:
    data = json.load(f)

# Convert to pandas DataFrame
df = pd.DataFrame(data)

# Create a Dataset object
dataset = Dataset.from_pandas(df)


In [4]:
from transformers import BartTokenizer

# Load the tokenizer
tokenizer = BartTokenizer.from_pretrained('facebook/bart-base')

# Tokenize the dataset
def preprocess_function(examples):
    inputs = examples["Question"]
    targets = examples["Query"]
    model_inputs = tokenizer(inputs, max_length=512, truncation=True)

    # Setup the tokenizer for targets
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(targets, max_length=512, truncation=True)

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

tokenized_dataset = dataset.map(preprocess_function, batched=True)


c:\Users\Prathiksha\AppData\Local\Programs\Python\Python310\lib\site-packages\huggingface_hub\file_download.py:159: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Prathiksha\.cache\huggingface\hub\models--facebook--bart-base. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
Map:   0%|          | 0/400 [00:00<?, ? examples/s]c:\Users\Prathiksha\AppData\Lo

In [5]:
from transformers import BartForConditionalGeneration, Trainer, TrainingArguments, DataCollatorForSeq2Seq

# Load the model
model = BartForConditionalGeneration.from_pretrained('facebook/bart-base')

# Define training arguments
training_args = TrainingArguments(
    output_dir='./results',
    eval_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    num_train_epochs=3,
    weight_decay=0.01,
)

# Create a data collator
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

# Initialize the Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,
    eval_dataset=tokenized_dataset,
    data_collator=data_collator,
)

# Train the model
trainer.train()


The installed version of bitsandbytes was compiled without GPU support. 8-bit optimizers, 8-bit multiplication, and GPU quantization are unavailable.
                                                   
 33%|███▎      | 200/600 [07:51<15:54,  2.39s/it]

{'eval_loss': 0.015744220465421677, 'eval_runtime': 67.9537, 'eval_samples_per_second': 5.886, 'eval_steps_per_second': 2.943, 'epoch': 1.0}


                                                   
 67%|██████▋   | 400/600 [16:33<05:56,  1.78s/it]

{'eval_loss': 0.009382620453834534, 'eval_runtime': 51.4324, 'eval_samples_per_second': 7.777, 'eval_steps_per_second': 3.889, 'epoch': 2.0}


 83%|████████▎ | 500/600 [32:07<02:50,  1.71s/it]    Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'early_stopping': True, 'num_beams': 4, 'no_repeat_ngram_size': 3, 'forced_bos_token_id': 0, 'forced_eos_token_id': 2}


{'loss': 0.2156, 'grad_norm': 3.7965614795684814, 'learning_rate': 3.3333333333333333e-06, 'epoch': 2.5}


100%|██████████| 600/600 [35:41<00:00,  2.60s/it]Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'early_stopping': True, 'num_beams': 4, 'no_repeat_ngram_size': 3, 'forced_bos_token_id': 0, 'forced_eos_token_id': 2}
                                                 
100%|██████████| 600/600 [36:49<00:00,  3.68s/it]

{'eval_loss': 0.007402641698718071, 'eval_runtime': 45.588, 'eval_samples_per_second': 8.774, 'eval_steps_per_second': 4.387, 'epoch': 3.0}
{'train_runtime': 2209.5627, 'train_samples_per_second': 0.543, 'train_steps_per_second': 0.272, 'train_loss': 0.18131971647342046, 'epoch': 3.0}


TrainOutput(global_step=600, training_loss=0.18131971647342046, metrics={'train_runtime': 2209.5627, 'train_samples_per_second': 0.543, 'train_steps_per_second': 0.272, 'total_flos': 9991579852800.0, 'train_loss': 0.18131971647342046, 'epoch': 3.0})

In [6]:
# Save the model
model.save_pretrained("./fine-tuned-bart")
tokenizer.save_pretrained("./fine-tuned-bart")


Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'early_stopping': True, 'num_beams': 4, 'no_repeat_ngram_size': 3, 'forced_bos_token_id': 0, 'forced_eos_token_id': 2}


('./fine-tuned-bart\\tokenizer_config.json',
 './fine-tuned-bart\\special_tokens_map.json',
 './fine-tuned-bart\\vocab.json',
 './fine-tuned-bart\\merges.txt',
 './fine-tuned-bart\\added_tokens.json')

In [7]:
from transformers import BartTokenizer, BartForConditionalGeneration

# Load the fine-tuned model and tokenizer
model = BartForConditionalGeneration.from_pretrained('./fine-tuned-bart')
tokenizer = BartTokenizer.from_pretrained('./fine-tuned-bart')

# Function to generate MongoDB query
def generate_query(question):
    inputs = tokenizer(question, return_tensors='pt', max_length=512, truncation=True, padding="max_length")
    outputs = model.generate(**inputs, max_length=512, num_beams=4, early_stopping=True)
    query = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return query

# Example usage
question = "What is the address of John Smith?"
query = generate_query(question)
print(query)


db.patients.find({"name.first": "John", "name.last": "Smith" }, { "contact.address": 1, "_id": 0 })


In [8]:
!pip install nltk rouge-score


In [9]:
import nltk
from nltk.translate.bleu_score import sentence_bleu
from rouge_score import rouge_scorer
from transformers import BartTokenizer, BartForConditionalGeneration

# Load the fine-tuned model and tokenizer
model = BartForConditionalGeneration.from_pretrained('./fine-tuned-bart')
tokenizer = BartTokenizer.from_pretrained('./fine-tuned-bart')


In [10]:
def generate_query_finetuned(question):
    inputs = tokenizer(question, return_tensors='pt', max_length=512, truncation=True, padding="max_length")
    outputs = model.generate(**inputs, max_length=512, num_beams=4, early_stopping=True)
    query = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return query


In [11]:
test_data = [

    {'input': 'Which patients are scheduled for surgery tomorrow?', 'expected_query': '{"surgery_date": "2024-07-29"}'},
    {'input': 'What appointments does Grace Smith have scheduled?', 'expected_query': '{"db.patients.find({ "name.first": "Grace", "name.last": "Smith" }, { "appointments": 1, "_id": 0"}'},
    {'input': 'What surgeries has Charlie Smith had in the past?', 'expected_query': '{"db.patients.find({ "name.first": "Charlie", "name.last": "Smith" }, { "medical_history.surgeries": 1, "_id": 0"}'},
    {'input': 'What is the insurance provider for David Smith?', 'expected_query': '{"db.patients.find({ "name.first": "David", "name.last": "Smith" }, { "insurance.provider": 1, "_id": 0"}'},
    {'input': 'What is the marital status of Charlie Smith?', 'expected_query': '{"db.patients.find({ "name.first": "Charlie", "name.last": "Smith" }, { "marital_status": 1, "_id": 0"}'},
    {'input': 'What is the contact phone number for Eve Smith?', 'expected_query': '{"db.patients.find({ "name.first": "Eve", "name.last": "Smith" }, { "contact.phone": 1, "_id": 0"}'},
    {'input': 'What surgeries has Olivia Smith had in the past?', 'expected_query': '{"db.patients.find({ "name.first": "Olivia", "name.last": "Smith" }, { "medical_history.surgeries": 1, "_id": 0"}'}
]


In [12]:
# Initialize metrics
bleu_scores = []
rouge_scores = {'rouge1': [], 'rouge2': [], 'rougeL': []}
precisions = []

# Initialize the ROUGE scorer
scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)


In [13]:
!pip install nltk rouge-score
import nltk
from nltk.translate.bleu_score import sentence_bleu
from rouge_score import rouge_scorer

# Download the punkt tokenizer models
nltk.download('punkt')



[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Prathiksha\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt.zip.


True

In [14]:
# Evaluation
for data in test_data:
    generated_query = generate_query_finetuned(data['input'])

    # Calculate BLEU score
    reference = nltk.word_tokenize(data['expected_query'])
    hypothesis = nltk.word_tokenize(generated_query)
    bleu_score = sentence_bleu([reference], hypothesis)
    bleu_scores.append(bleu_score)

    # Calculate ROUGE scores
    rouge_score = scorer.score(data['expected_query'], generated_query)
    rouge_scores['rouge1'].append(rouge_score['rouge1'].fmeasure)
    rouge_scores['rouge2'].append(rouge_score['rouge2'].fmeasure)
    rouge_scores['rougeL'].append(rouge_score['rougeL'].fmeasure)

    # Calculate Precision
    reference_set = set(reference)
    hypothesis_set = set(hypothesis)
    true_positives = len(reference_set.intersection(hypothesis_set))
    precision = true_positives / len(hypothesis_set) if hypothesis_set else 0
    precisions.append(precision)


c:\Users\Prathiksha\AppData\Local\Programs\Python\Python310\lib\site-packages\nltk\translate\bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)


In [15]:
# Average metrics
avg_bleu = sum(bleu_scores) / len(bleu_scores)
avg_rouge1 = sum(rouge_scores['rouge1']) / len(rouge_scores['rouge1'])
avg_rouge2 = sum(rouge_scores['rouge2']) / len(rouge_scores['rouge2'])
avg_rougeL = sum(rouge_scores['rougeL']) / len(rouge_scores['rougeL'])
avg_precision = sum(precisions) / len(precisions)

print(f'Average BLEU Score: {avg_bleu}')
print(f'Average ROUGE-1 Score: {avg_rouge1}')
print(f'Average ROUGE-2 Score: {avg_rouge2}')
print(f'Average ROUGE-L Score: {avg_rougeL}')
print(f'Average Precision: {avg_precision}')


Average BLEU Score: 0.7238242728728396
Average ROUGE-1 Score: 0.8568067226890757
Average ROUGE-2 Score: 0.8260869565217391
Average ROUGE-L Score: 0.8568067226890757
Average Precision: 0.8532212885154064


In [18]:
import joblib

joblib.dump(model, 'model.joblib')
joblib.dump(tokenizer, 'tokenizer.joblib')

['tokenizer.joblib']